# Fase 2: Extracción de Detalles de Películas con Selenium
Una vez que tengan una lista de películas de la API, el siguiente paso es obtener información más detallada sobre ellas. Utilizaras la herramienta de automatización web Selenium para navegar a sitios web de reseñas de películas (IMDB y Rotten Tomatoes) y extraer detalles como calificaciones, actores y directores. En concreto deberás extraer información sobre:
- Puntuacion de IMDB (en caso de que la tenga).
- Puntuación de Rotten Tomatoes (Tomatometer).
- Dirección (director/a o directore/as de cada película).
- Guionistas (de cada película).
- Argumento.
- Duración (en minutos).
- Nombre de la película
NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. Siguiendo el siguiente ejemplo:

[(7.7, 77,  "Richard Donner", ["Chris ColumbusSteven", "Spielberg"], "Los Goonies son un grupo de amigos que viven en Goon Docks, Astoria, pero sus casas han sido compradas y van a ser demolidas. Sin embargo, vivirán su última aventura en busca de un tesoro que pueda salvar el barrio.", "Aventura", "1h 54min", "Los Gonnies"),  ...]

In [1]:
!pip install beautifulsoup4

!pip install selenium
!pip install webdriver-manager

!pip install openpyxl


In [10]:
# Importar librerías para tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd  # Pandas es una poderosa librería para manipulación y análisis de datos en Python.

# Importar librerías para web scraping y manipulación de datos
# -----------------------------------------------------------------------
from bs4 import BeautifulSoup
import requests

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.

# Importar librerías para pausar la ejecución
import time 
from time import sleep
 # Sleep se utiliza para pausar la ejecución del programa por un número de segundos.
from tqdm import tqdm
# Configuraciones
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

import re

import openpyxl



In [13]:
file = pd.read_excel('Thriller.xlsx')
df = pd.DataFrame(file)


In [14]:
#Abrimos el Chrome
driver = webdriver.Chrome()

# Maximizar la ventana del navegador
driver.maximize_window()

lista_info_pelis_final = []

#Introducimos id_peli y se busca
for id_peli in tqdm(df[5]):

    # Navegar a la página web de IMDB 
    driver.get("https://www.imdb.com")

    #Quitamos lo de la cuenta

    try:
        cuenta=driver.find_element('css selector','#imdbHeader > div.ipc-page-content-container.ipc-page-content-container--center.navbar__inner > div.nav__userMenu.navbar__user.sc-jfTVlA.esWGB > div > div > div > div > div.sc-cOxWqc.bNDQpU > button > svg').click()
        #id_peli='tt0075259' #Ejemplo, luego habrá que hacer bucles for :( 
    except:
        pass
    lista_info_peli = []

    busqueda_peli = driver.find_element('css selector', '#suggestion-search').send_keys(id_peli, Keys.ENTER)

    puntuacion_imdb = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-3a4309f8-0.fjtZsE.sc-dffc6c81-1.fJrHDo > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.gtEgaf > div.sc-bde20123-2.gYgHoj > span.sc-bde20123-1.iZlgcd').text
    
    sleep(3)
    direccion = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div > ul').text
    direccion.split("\n")
                                                  
    guion = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > div > section > div > div.sc-9178d6fe-1.kFxVZc.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.gmonkL.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div > ul').text
  

    argumento = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[1]/section/p/span[3]').text
  

    try:
        duracion = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(3)').text
    except:
        
        try:
            duracion  =driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > ul > li:nth-child(2)').text
        except:
            duracion = 'None'
    
  

    sleep(2)

    #Podemos cogerlo también del dataFrame, igual que el id
    titulo = driver.find_element('css selector','#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.eaRXHu > section > div:nth-child(4) > section > section > div.sc-e226b0e3-3.jJsEuz > div.sc-dffc6c81-0.iwmAVw > h1').text
  

    sleep(2)
    # Navegar a la página web de en Rottentomatoes 
    driver.get("https://www.rottentomatoes.com/")

    #Cerrar cookies
    try:
        cookies = driver.find_element('css selector','#onetrust-accept-btn-handler').click()
    except:
        pass

    #Buscar peli 
    #titulo_peli='Star Wars: Episode VII - The Force Awakens' #Ejemplo, luego habrá que hacer bucles for :( 
    busqueda_peli_tomates=driver.find_element('css selector', '#header-main > search-algolia > search-algolia-controls > input').send_keys(titulo, Keys.ENTER)

    driver.execute_script('window.scrollTo(0,300)')

    sleep(3)
    #Capturar la url de la peli
    try:
        url_peli = driver.find_element("css selector", '#search-results > search-page-result:nth-child(3) > ul > search-page-media-row > a:nth-child(2)').get_attribute("href")
        #Entrar en la peli
        entrar_peli=driver.find_element('css selector','#search-results > search-page-result:nth-child(3) > ul > search-page-media-row:nth-child(1) > a:nth-child(2)').click()
        sleep(2)
        #Beautifulsoup de la peli
        respues_peli = requests.get(url_peli)

        respues_peli.status_code

        sopa_peli = BeautifulSoup(respues_peli.content,"html.parser")

        try:
            tomate = str(sopa_peli.findAll('score-board-deprecated')[0])
            #Regex para capturar tomatometerscore
            tomatometer_score = re.findall('tomatometerscore="\d+', tomate)
            #Regex para capturar el valor
            puntuacion_tomate = re.findall("\d+", tomatometer_score[0])

        except:
            puntuacion_tomate = "None"
    except:
        puntuacion_tomate = "None"
   
    lista_info_peli.append(puntuacion_imdb)
    lista_info_peli.append(puntuacion_tomate[0])
    lista_info_peli.append(direccion)
    lista_info_peli.append(guion)
    lista_info_peli.append(argumento)
    lista_info_peli.append(duracion)
    lista_info_peli.append(titulo)

    lista_info_pelis_final.append(tuple(lista_info_peli))

driver.close()

print(lista_info_pelis_final)


  0%|          | 4/25328 [01:50<194:06:14, 27.59s/it]


KeyboardInterrupt: 

In [ ]:
df_lista_final = pd.DataFrame(lista_info_pelis_final)
df_lista_final.to_excel("Thriller_final.xlsx")

In [6]:
file = pd.read_excel('peliculasmisterio.xlsx')
df = pd.DataFrame(file)
df[5]

for id in df[5]:
    id


0      tt0075259
1      tt0092589
2      tt0094997
3      tt0096177
4      tt0097573
         ...    
117    tt5790428
118    tt6244888
119    tt6788638
120    tt8422674
121    tt8119162
Name: 5, Length: 122, dtype: object